## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,80.15,73,9,13.24,light rain
1,1,San Patricio,US,28.0170,-97.5169,70.07,82,100,16.06,overcast clouds
2,2,Arlit,NE,18.7369,7.3853,75.79,14,73,9.93,broken clouds
3,3,Okha,RU,53.5739,142.9478,19.13,75,12,20.07,few clouds
4,4,Spijkenisse,NL,51.8450,4.3292,39.02,78,0,2.30,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,80.15,73,9,13.24,light rain
2,2,Arlit,NE,18.7369,7.3853,75.79,14,73,9.93,broken clouds
6,6,Flinders,AU,-34.5833,150.8552,75.25,63,47,7.52,scattered clouds
9,9,Makakilo City,US,21.3469,-158.0858,76.28,74,75,9.22,broken clouds
10,10,Nizwa,OM,22.9333,57.5333,79.34,17,0,1.97,clear sky
11,11,Am Timan,TD,11.0297,20.2827,77.20,12,85,5.93,overcast clouds
14,14,Ormara,PK,25.2088,64.6357,76.37,70,0,4.05,clear sky
17,17,Cockburn Town,TC,21.4612,-71.1419,77.41,75,0,14.90,clear sky
20,20,Souillac,MU,-20.5167,57.5167,77.38,100,75,0.00,shower rain
21,21,Avarua,CK,-21.2078,-159.7750,82.45,78,20,12.66,few clouds


In [9]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,80.15,light rain,-23.1203,-134.9692,
2,Arlit,NE,75.79,broken clouds,18.7369,7.3853,
6,Flinders,AU,75.25,scattered clouds,-34.5833,150.8552,
9,Makakilo City,US,76.28,broken clouds,21.3469,-158.0858,
10,Nizwa,OM,79.34,clear sky,22.9333,57.5333,
11,Am Timan,TD,77.20,overcast clouds,11.0297,20.2827,
14,Ormara,PK,76.37,clear sky,25.2088,64.6357,
17,Cockburn Town,TC,77.41,clear sky,21.4612,-71.1419,
20,Souillac,MU,77.38,shower rain,-20.5167,57.5167,
21,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)
# 7. Drop the rows where there is no Hotel Name.

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,80.15,light rain,-23.1203,-134.9692,People ThankYou
2,Arlit,NE,75.79,broken clouds,18.7369,7.3853,Case de Passage De la Prefecture
6,Flinders,AU,75.25,scattered clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
9,Makakilo City,US,76.28,broken clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
10,Nizwa,OM,79.34,clear sky,22.9333,57.5333,Orient Apartments
11,Am Timan,TD,77.20,overcast clouds,11.0297,20.2827,Palais du chef Canton Salamt
14,Ormara,PK,76.37,clear sky,25.2088,64.6357,Kenara Colony house for Rent
17,Cockburn Town,TC,77.41,clear sky,21.4612,-71.1419,Osprey Beach Hotel
20,Souillac,MU,77.38,shower rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
21,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,Paradise Inn


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))